In [15]:
import pandas as pd

In [16]:
def ToD(df,m):
    df['{}'.format(m)]=pd.to_datetime(df['{}'.format(m)],format='%d/%m/%Y %H:%M:%S') 
    df['Month']=df['{}'.format(m)].dt.month # monthly information 
    df['CY']=df['{}'.format(m)].dt.year    # calander year information 
    df['Quarter']=df['{}'.format(m)].dt.quarter# quartely information
    df['FY']=df['{}'.format(m)].map(lambda d: d.year + 1 if d.month > 6 else d.year) # Financial year infromation
    df['Date']=df['{}'.format(m)].dt.date

In [17]:
df_d=pd.read_csv(r'D:\others_Output\High Price\Raw Data\NEM Registration and Exemption List_DUID_INFO.csv')

In [18]:
colnames=['SETTLEMENTDATE', 'MARKET', 'PARTICIPANT', 'UNIT','DISPATCHEDEMAREKT','Spot Price','RRN Offer Price',' Increase','other1','other2','other3','other4','other5','other6','other7'] 

In [19]:
df=pd.read_csv('D:\others_Output\High Price\Raw Data\QLD1_5_Min_2024_Price Setter.csv', skiprows=1, names=colnames, header=None)

C:\Users\ygu\AppData\Local\Temp\ipykernel_23924\3150329625.py:1: DtypeWarning: Columns (8,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('D:\others_Output\High Price\Raw Data\QLD1_5_Min_2024_Price Setter.csv', skiprows=1, names=colnames, header=None)


In [20]:
df=df[(df['other2'] !='tbslack1') & (df['other2'] !='tbslack2')]
df=df[(df['other3'] !='tbslack1') & (df['other3'] !='tbslack2')]
df=df[df['DISPATCHEDEMAREKT']=='ENOF']


In [21]:
df_d=df_d[['Fuel Source - Primary','DUID']].copy()

In [22]:
df['DISPATCHEDEMAREKT'].unique()

array(['ENOF'], dtype=object)

In [23]:
df=df.merge(df_d, left_on=['UNIT'], right_on=['DUID'], how='left')
df.drop(columns=['other1','other2','other3','other4','other5','other6','other7'], inplace=True)
df=df.drop_duplicates(subset=['SETTLEMENTDATE','PARTICIPANT'], keep='first')
df=df[['SETTLEMENTDATE','Fuel Source - Primary','Spot Price','RRN Offer Price',' Increase']].copy()

In [36]:
df=df.rename(columns={' Increase':'Increase'})

In [37]:
df.columns

Index(['SETTLEMENTDATE', 'Fuel Source - Primary', 'Spot Price',
       'RRN Offer Price', 'Increase', 'Month', 'CY', 'Quarter', 'FY', 'Date'],
      dtype='object')

In [38]:
df.columns

Index(['SETTLEMENTDATE', 'Fuel Source - Primary', 'Spot Price',
       'RRN Offer Price', 'Increase', 'Month', 'CY', 'Quarter', 'FY', 'Date'],
      dtype='object')

In [28]:
ToD(df,'SETTLEMENTDATE')

In [42]:
df['RRN Offer Price']=df['RRN Offer Price'].astype(float)
df['Increase']=df['Increase'].astype(float)
df['Spot Price']=df['Spot Price'].astype(float)

In [45]:
df.groupby(['Date']).agg({'Spot Price':'mean'})

,Spot Price
Date,
2024-01-01,38.721748
2024-01-02,39.899275
2024-01-03,55.647981
2024-01-04,62.058848
2024-01-05,50.692301
...,...
2024-12-27,41.123048
2024-12-28,59.869967
2024-12-29,39.957043


In [49]:
df.groupby(['Date','Fuel Source - Primary']).agg({'Increase':['sum', 'mean'],'RRN Offer Price':['mean']})

Increase           RRN Offer Price
                                      sum      mean            mean
Date       Fuel Source - Primary                                   
2024-01-01 Fossil                  125.22  0.433287       45.503529
           Hydro                    13.21  0.440333       50.715333
           Solar                    12.91  0.461071     -193.197143
2024-01-02 Fossil                   93.88  0.303819       49.344207
           Hydro                    36.83  0.920750       58.072250
...                                   ...       ...             ...
2024-12-30 Wind                      3.07  1.023333      -33.310000
2024-12-31 Fossil                   77.03  0.531241       91.791379
           Hydro                    56.94  1.035273      122.038000
           Solar                    59.26  0.740750      -91.115875
           Wind                     -0.04 -0.020000      -48.105000

[1398 rows x 3 columns]

In [12]:
df1=df.groupby(['Fuel Source - Primary','Date']).agg({'RRN Offer Price':'mean'}).reset_index()

In [13]:
#df1.to_csv('D:\others_Output\High Price\Input\QLD_2024.csv')

In [14]:
df

,SETTLEMENTDATE,Fuel Source - Primary,Spot Price,RRN Offer Price,Month,CY,Quarter,FY,Date
0,2024-01-01 00:05:00,Hydro,65.49,57.98,1,2024,1,2024,2024-01-01
1,2024-01-01 00:10:00,Hydro,79.41,85.28,1,2024,1,2024,2024-01-01
2,2024-01-01 00:15:00,Hydro,65.48,57.98,1,2024,1,2024,2024-01-01
3,2024-01-01 00:20:00,Fossil,61.6,54.95,1,2024,1,2024,2024-01-01
6,2024-01-01 00:25:00,Fossil,61.54,54.95,1,2024,1,2024,2024-01-01
...,...,...,...,...,...,...,...,...,...
252635,2024-12-31 23:05:00,Hydro,164.5,164.50,12,2024,4,2025,2024-12-31
252636,2024-12-31 23:10:00,Hydro,164.5,164.50,12,2024,4,2025,2024-12-31
252637,2024-12-31 23:15:00,Hydro,214.48,214.48,12,2024,4,2025,2024-12-31
252638,2024-12-31 23:45:00,Hydro,160.04,130.00,12,2024,4,2025,2024-12-31


In [95]:
df.groupby(['Date','Fuel Source - Primary']).agg({'RRN Offer Price':'mean'}).reset_index()

,Date,Fuel Source - Primary,RRN Offer Price
0,2024-01-01,Fossil,45.503529
1,2024-01-01,Hydro,50.715333
2,2024-01-01,Solar,-193.197143
3,2024-01-02,Fossil,49.344207
4,2024-01-02,Hydro,58.072250
...,...,...,...
1393,2024-12-30,Wind,-33.310000
1394,2024-12-31,Fossil,91.791379
1395,2024-12-31,Hydro,122.038000
1396,2024-12-31,Solar,-91.115875


,SETTLEMENTDATE,MARKET,PARTICIPANT,UNIT,DISPATCHEDEMAREKT,Spot Price,RRN Offer Price,Increase,Fuel Source - Primary,DUID,Month,CY,Quarter,FY,Date
0,2024-01-01 00:05:00,Energy,SNOWY,UPPTUMUT,ENOF,65.49,57.98,1.13,Hydro,UPPTUMUT,1,2024,1,2024,2024-01-01
1,2024-01-01 00:10:00,Energy,HYDROTAS,GORDON,ENOF,79.41,85.28,1.15,Hydro,GORDON,1,2024,1,2024,2024-01-01
2,2024-01-01 00:15:00,Energy,SNOWY,UPPTUMUT,ENOF,65.48,57.98,1.13,Hydro,UPPTUMUT,1,2024,1,2024,2024-01-01
3,2024-01-01 00:20:00,Energy,OERARING,ER01,ENOF,61.6,54.95,0.37,Fossil,ER01,1,2024,1,2024,2024-01-01
6,2024-01-01 00:25:00,Energy,OERARING,ER01,ENOF,61.54,54.95,0.56,Fossil,ER01,1,2024,1,2024,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252635,2024-12-31 23:05:00,Energy,CLEANCO,W/HOE#2,ENOF,164.5,164.5,1,Hydro,W/HOE#2,12,2024,4,2025,2024-12-31
252636,2024-12-31 23:10:00,Energy,CLEANCO,W/HOE#2,ENOF,164.5,164.5,1,Hydro,W/HOE#2,12,2024,4,2025,2024-12-31
252637,2024-12-31 23:15:00,Energy,CLEANCO,W/HOE#2,ENOF,214.48,214.48,1,Hydro,W/HOE#2,12,2024,4,2025,2024-12-31
252638,2024-12-31 23:45:00,Energy,SNOWY,MURRAY,ENOF,160.04,130,1.23,Hydro,MURRAY,12,2024,4,2025,2024-12-31
